## 1. 파이썬 모듈 불러들이기

In [1]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize(): 
    np.random.seed(time.time())

## 2. 하이퍼파라미터값의 정의

In [2]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

## 3. 실험용 메인 함수 정의

In [3]:
def abalone_exec(epoch_count = 10, mb_size = 10, report=1): # (학습 횟수, 미니배치크기, 보고서작성주기)
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

## 4. 데이터 적재 함수 정의

In [5]:
def load_abalone_dataset():
    with open('abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None) # 헤더 행을 무시하기 위함. None은 반복 가능 객체의 마지막 요소 이후 반환할 기본값
        rows = []
        for row in csvreader:
            rows.append(row)
            
        global data, input_cnt, output_cnt # 다른 함수에서도 input벡터의 수와 output 벡터의 수를 사용하기 위함
        input_cnt, output_cnt = 10, 1
        data = np.zeros([len(rows), input_cnt+output_cnt]) # 전복의 성별 정보를 원핫벡터 표현으로 변환하고 나머지 정보를 복사하기 위함
    
        for n, row in enumerate(rows): 
            if row[0] == 'I': data[n, 0] = 1
            if row[0] == 'M': data[n, 1] = 1
            if row[0] == 'F': data[n, 2] = 1
            data[n, 3:] = row[1:]

## 5. 파라미터 초기화 함수 정의

In [6]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt]) # 편향은 초기에 지나친 영향을 주어 학습에 역효과 없도록 0으로 초기화하여 생성

## 6. 학습 및 평가 함수 정의

In [7]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size) # 훈련용 데이터수를 미니배치로 나눈, 미니배치 반복횟수
    test_x, test_y = get_test_data() # 테스트용 데이터셋은 변하지 않고 반복적으로 이용되므로 맨 처음 한 번만 호출함
    
    for epoch in range(epoch_count): # 학습 반복횟수
        losses, accs = [], []
        
        for n in range(step_count): 
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {} loss={:5.3f}. accuracy = {:5.3f}/{:5.3f}'. # 전체 5글자 소수점이하 3글자
                 format(epoch + 1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

## 7. 학습 및 평가 데이터 획득 함수 정의

In [9]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0]) # 데이터 수만큼의 일련번호를 발생시킴
    np.random.shuffle(shuffle_map) # 무작위로 순서를 섞음
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]] # 무작위로 섞인 일련번호의 마지막 20% (테스트 데이터부분)을 추출함
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:] # test_x와 test_y로 나누어 반환, 자동으로 튜플로 묶임

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, ouput_cnt
    if nth == 0: 
        np.random.shuffle(shuffle_map[:test_begin_idx]) # get_test_data 함수와 마찬가지로 처음에 일련번호를 무작위로 섞음
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]] # n번째 (nth) 훈련용 데이터 mb_size 개를 추출함
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:] # train_x와 train_y로 나누어 반환

## 8. 학습 실행 함수와 평가 실행 함수 정의

In [10]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    # aux_nn, aux_pp(neuralnet, postproc는 역전파에 필요한 정보 가운데 순전파 과정에서만 확보 가능하거나 보다 효율적으로 확보 가능한 것들
    # forward_postproc은 output과 정답 y를 이용해 회귀 분석 문제의 성격에 맞춘 후처리 순전파작업 수행(순전파 후처리-postprocess)
    accuracy = eval_accuracy(output, y) # 정확도
    
    G_loss = 1.0 # 손실 기울기, 처음엔 자기 자신으로 미분하므로 1
    G_output = backprop_postproc(G_loss, aux_pp) # 순전파의 역순이므로 이름은 후처리지만 먼저 처리함
    backprop_neuralnet(G_output, aux_nn) # 신경망 파라미터의 변화, 즉 학습이 실제로 일어나는 부분
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

## 9. 단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의

In [ ]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x # 역전파를 위해 이용될 보조 정보로 x도 함께 반환 

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose() # 가중치의 손실 기울기 구하는 과정에서 필요 (80p)
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis = 0) # B_j는 Y_1j, Y_2j, ... Y_mj모두를 통해 L값에 영향을 미침
    
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

## 10. 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [ ]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff): #s 순전파: 편차 -> 제곱 -> 평균/역전파: 평균 -> 제곱 ->편차
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape) # 평균 연산에 대한 미분값, 1 / M * N, *np.prod(shape): shape의 모든 값을 곱함
    g_square_diff = 2 * diff # square_ij = diff_ij ** 2이므로
    g_diff_output = 1 # diff_ij = output_ij - y_ij이므로
     
    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

## 10-1. backprop_postproc 함수 한 줄 코드

In [9]:
def backprop_postproc_oneline(G_loss, diff):
    return 2 * diff / np.prrod(diff.shape) # G_loss, g_diff_output, np.ones(shape)가 모두 1이므로

## 11. 정확도 계산 함수 정의

In [10]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y) / y)) # 정답과 오차의 비율, 오류율, 회귀 분석 문제에서 정확도를 정의하는 다른 시각도 있음
    return 1 - mdiff